<a href="https://colab.research.google.com/github/TarunKumar19/TrafficSignsClassification/blob/main/Traffic_Signs_Classification_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')